# Сборный проект - 2

**Описание проекта**

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 

Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

Напишем запрос на выгрузку всех городов, для проверки их валидности.

In [3]:
query = """
SELECT city
FROM location 
"""

Видим, что есть пять городов со слипшимися названиями - "НабережныеЧелны, НижнийНовгород, НижнийТагил, РостовнаДону, СанктПетербург".

В главном запросе скоректируем названия этих городов

In [4]:
df = pd.read_sql(query, engine)
set(df['city'])

{'Архангельск',
 'Астрахань',
 'Балашиха',
 'Барнаул',
 'Белгород',
 'Брянск',
 'Владивосток',
 'Владимир',
 'Волгоград',
 'Волжский',
 'Воронеж',
 'Грозный',
 'Екатеринбург',
 'Иваново',
 'Ижевск',
 'Иркутск',
 'Казань',
 'Калининград',
 'Калуга',
 'Кемерово',
 'Киров',
 'Краснодар',
 'Красноярск',
 'Курск',
 'Липецк',
 'Магнитогорск',
 'Махачкала',
 'Москва',
 'НабережныеЧелны',
 'НижнийНовгород',
 'НижнийТагил',
 'Новокузнецк',
 'Новосибирск',
 'Омск',
 'Оренбург',
 'Пенза',
 'Пермь',
 'РостовнаДону',
 'Рязань',
 'Самара',
 'СанктПетербург',
 'Саранск',
 'Саратов',
 'Смоленск',
 'Сочи',
 'Ставрополь',
 'Сургут',
 'Тверь',
 'Тольятти',
 'Томск',
 'Тула',
 'Тюмень',
 'УланУдэ',
 'Ульяновск',
 'Уфа',
 'Хабаровск',
 'Чебоксары',
 'Челябинск',
 'Череповец',
 'Чита',
 'Якутск',
 'Ярославль'}

Посмотрим на число записей в таблице user перед составлением витрины, для того чтобы убедиться, что все записи попали в нашу выборку.

In [5]:
query = """
SELECT count(*)
FROM user 
"""

Количество записей: 502493

In [6]:
df = pd.read_sql(query, engine)
df.head()

,count(*)
0,502493


In [7]:
query = """
SELECT u.user_id, 
    u.lt_day, 
    CASE
        WHEN u.lt_day <= 365 THEN 'new'
        ELSE 'old'
    END AS is_new, 
    u.age, u.gender_segment, u.os_name, u.cpe_type_name, l.country, 
    CASE
        WHEN l.city = 'РостовнаДону' THEN 'Ростов-на-Дону'
        WHEN l.city = 'СанктПетербург' THEN 'Санкт-Петербург'
        WHEN l.city = 'НабережныеЧелны' THEN 'Набережные Челны'
        WHEN l.city = 'НижнийНовгород' THEN 'Нижний Новгород'
        WHEN l.city = 'НижнийТагил' THEN 'Нижний Тагил'
        ELSE l.city
    END AS city, 
    SUBSTR(ag.title, 4) AS age_segment,  
    SUBSTR(tr.title, 4) AS traffic_segment, 
    SUBSTR(li.title, 4) AS lifetime_segment, 
    u.nps_score, 
    CASE
        WHEN u.nps_score >= 9 THEN 'cторонники'
        WHEN u.nps_score >= 7 THEN 'нейтралы'
        ELSE 'критики'
    END AS nps_group
FROM user u
INNER JOIN location l ON u.location_id = l.location_id
INNER JOIN age_segment ag ON u.age_gr_id = ag.age_gr_id
INNER JOIN traffic_segment tr ON u.tr_gr_id = tr.tr_gr_id
INNER JOIN lifetime_segment li ON u.lt_gr_id = li.lt_gr_id;
"""

In [8]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,old,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,old,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники
3,A004TB,4190,old,44.0,1.0,IOS,SMARTPHONE,Россия,Ростов-на-Дону,35-44,0.1-1,36+,10,cторонники
4,A004XT,1163,old,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонники
5,A005O0,5501,old,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,old,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,cторонники
7,A009KS,313,new,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,cторонники
8,A00AES,3238,old,36.0,1.0,ANDROID,SMARTPHONE,Россия,Санкт-Петербург,35-44,1-5,36+,10,cторонники
9,A00F70,4479,old,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,cторонники


Количество записей также 502493

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  float64
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 53.7+ MB


In [10]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Создание дашборда в Tableau**

https://public.tableau.com/app/profile/.36841714/viz/-2_16976652680360/sheet23?publish=yes